In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import joblib

In [2]:
final_rmse = 5000

In [3]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [4]:
def prepare_data(df, sequence_length):
    df = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values #optional
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [5]:
def build_gru_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(GRU(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(GRU(units, return_sequences=True))
    model.add(GRU(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [6]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    gru_loss_history = loss_history.history['loss'] # unnecessary line
    
    gru_predictions = model.predict(X_test)
    gru_predictions = scaler.inverse_transform(gru_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, gru_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "BCH_model_gru.pkl")
        
        plt.plot(y_test, label='Actual')
        plt.plot(gru_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'BCH_actual_vs_predicted_GRU.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'BCH_loss_curve_GRU.png')
        plt.close()
    
    return model, rmse


In [7]:
symbol = 'BCH-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [8]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,20]
units_range = [50, 100]
layers_range = [2, 3]

In [9]:
results = []

In [10]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range: #[50, 100]
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_gru_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - 17s 34ms/step - loss: 0.0026 - val_loss: 1.5717e-05
Epoch 2/100
  3/268 [..............................] - ETA: 7s - loss: 2.2549e-04

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 7s 25ms/step - loss: 0.0010 - val_loss: 2.1172e-05
Epoch 3/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0012 - val_loss: 4.2607e-05
Epoch 4/100
268/268 [==============================] - 7s 25ms/step - loss: 9.7470e-04 - val_loss: 8.0175e-04
Epoch 5/100
268/268 [==============================] - 7s 25ms/step - loss: 8.2481e-04 - val_loss: 4.7924e-05
Epoch 6/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0010 - val_loss: 4.7075e-05
Epoch 7/100
268/268 [==============================] - 7s 26ms/step - loss: 7.6563e-04 - val_loss: 1.9392e-04
Epoch 8/100
268/268 [==============================] - 7s 26ms/step - loss: 8.5841e-04 - val_loss: 2.1505e-04
Epoch 9/100
268/268 [==============================] - 7s 27ms/step - loss: 8.4789e-04 - val_loss: 8.0729e-06
Epoch 10/100
268/268 [==============================] - 8s 31ms/step - loss: 7.3741e-04 - val_loss: 1.0675e-05
Epoch 11/100
268/268 [===========

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 9s 33ms/step - loss: 0.0012 - val_loss: 0.0017
Epoch 3/100
268/268 [==============================] - 9s 33ms/step - loss: 0.0011 - val_loss: 9.6274e-04
Epoch 4/100
268/268 [==============================] - 9s 33ms/step - loss: 0.0017 - val_loss: 1.8496e-04
Epoch 5/100
268/268 [==============================] - 9s 33ms/step - loss: 9.1269e-04 - val_loss: 4.2344e-04
Epoch 6/100
268/268 [==============================] - 9s 34ms/step - loss: 0.0012 - val_loss: 0.0018
Epoch 7/100
268/268 [==============================] - 9s 33ms/step - loss: 8.7246e-04 - val_loss: 1.7848e-04
Epoch 8/100
268/268 [==============================] - 9s 34ms/step - loss: 9.0928e-04 - val_loss: 2.3844e-05
Epoch 9/100
268/268 [==============================] - 9s 33ms/step - loss: 7.9112e-04 - val_loss: 3.5405e-04
Epoch 10/100
268/268 [==============================] - 9s 33ms/step - loss: 0.0012 - val_loss: 3.2595e-05
Epoch 11/100
268/268 [===========================

...

134/134 [==============================] - 4s 32ms/step - loss: 0.0016 - val_loss: 8.7817e-04
Epoch 3/100
134/134 [==============================] - 4s 31ms/step - loss: 0.0013 - val_loss: 3.0855e-05
Epoch 4/100
134/134 [==============================] - 4s 33ms/step - loss: 0.0013 - val_loss: 1.2547e-05
Epoch 5/100
134/134 [==============================] - 6s 41ms/step - loss: 9.4406e-04 - val_loss: 0.0014
Epoch 6/100
134/134 [==============================] - 5s 35ms/step - loss: 9.0031e-04 - val_loss: 3.7073e-04
Epoch 7/100
134/134 [==============================] - 4s 33ms/step - loss: 8.1993e-04 - val_loss: 0.0021
Epoch 8/100
134/134 [==============================] - 5s 34ms/step - loss: 8.9857e-04 - val_loss: 7.9563e-04
Epoch 9/100
134/134 [==============================] - 4s 33ms/step - loss: 9.8054e-04 - val_loss: 4.7455e-05
Epoch 10/100
134/134 [==============================] - 4s 33ms/step - loss: 7.0704e-04 - val_loss: 9.6201e-06
Epoch 11/100
134/134 [===================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 5s 37ms/step - loss: 0.0012 - val_loss: 7.8976e-05
Epoch 3/100
134/134 [==============================] - 5s 36ms/step - loss: 9.3564e-04 - val_loss: 8.4254e-04
Epoch 4/100
134/134 [==============================] - 5s 36ms/step - loss: 0.0010 - val_loss: 7.3648e-04
Epoch 5/100
134/134 [==============================] - 5s 37ms/step - loss: 8.3490e-04 - val_loss: 3.1647e-04
Epoch 6/100
134/134 [==============================] - 5s 36ms/step - loss: 8.6205e-04 - val_loss: 6.1639e-04
Epoch 7/100
134/134 [==============================] - 5s 36ms/step - loss: 9.9823e-04 - val_loss: 0.0012
Epoch 8/100
134/134 [==============================] - 5s 37ms/step - loss: 0.0011 - val_loss: 7.0195e-06
Epoch 9/100
134/134 [==============================] - 5s 36ms/step - loss: 8.0749e-04 - val_loss: 1.7867e-04
Epoch 10/100
134/134 [==============================] - 5s 36ms/step - loss: 9.8495e-04 - val_loss: 7.8074e-04
Epoch 11/100
134/134 [===============

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 6s 45ms/step - loss: 0.0019 - val_loss: 1.2887e-04
Epoch 3/100
134/134 [==============================] - 6s 41ms/step - loss: 0.0013 - val_loss: 2.4305e-04
Epoch 4/100
134/134 [==============================] - 6s 46ms/step - loss: 7.8231e-04 - val_loss: 8.8079e-04
Epoch 5/100
134/134 [==============================] - 6s 48ms/step - loss: 8.0873e-04 - val_loss: 1.2465e-04
Epoch 6/100
134/134 [==============================] - 6s 46ms/step - loss: 8.8859e-04 - val_loss: 8.7651e-06
Epoch 7/100
134/134 [==============================] - 6s 46ms/step - loss: 0.0012 - val_loss: 8.5369e-06
Epoch 8/100
134/134 [==============================] - 6s 45ms/step - loss: 8.6229e-04 - val_loss: 6.5466e-05
Epoch 9/100
134/134 [==============================] - 6s 46ms/step - loss: 0.0012 - val_loss: 7.5085e-04
Epoch 10/100
134/134 [==============================] - 6s 46ms/step - loss: 8.9050e-04 - val_loss: 5.5092e-05
Epoch 11/100
134/134 [===============

In [12]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 10.516443595185972}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 10.932642231243596}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 10.406238655525629}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 10.59071982924709}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 2, 'rmse': 9.632178224732975}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 3, 'rmse': 9.36574959062417}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 2, 'rmse': 11.241165129959544}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 3, 'rmse': 9.968086892611783}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 9.337277593510477}
{'epochs': 1